In [50]:
import urllib.request as req
from bs4 import BeautifulSoup as soup
import pandas as pd

In [3]:
#using browser to scrape
user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
headers={'User-Agent':user_agent,} 

In [8]:
#url for main site
main_url = "https://cookieandkate.com/"

#getting main site page info
request_main = req.Request(main_url, None, headers) #The assembled request
response_main = req.urlopen(request_main)
data_main = response_main.read()
response_main.close()

page_main = soup(data_main,"html.parser")

In [49]:
#retrieving categories of recipes
categories = pagemain.find("nav",{"id":"genesis-nav-primary"}).ul.findAll("li")[2].findAll("li")
category_links = []
for category in categories:
    category_links.append(category.a["href"])

In [54]:
#creating a list for all recipes
recipe_links = []

#looping through categories
for category_url in category_links:
    
    #getting first page info for current category
    category_request = req.Request(category_url, None, headers)
    category_response = req.urlopen(category_request)
    category_data = category_response.read()
    category_response.close()

    category_page = soup(category_data,"html.parser")
    
    while True:

        #next page marker
        next_marker = category_page.find("div", {"class":"pagination-next alignright"})

        #if there is a next page
        if next_marker:

            #get url for page
            next_url = next_marker.a['href']

            #read info from page
            next_request = req.Request(next_url, None, headers)
            next_response = req.urlopen(next_request)
            next_data = next_response.read()
            next_response.close()

            #set category_page to new page
            category_page = soup(next_data)

            #collect all recipes from new page
            recipes = category_page.findAll("div",{"class":"lcp_catlist_item"})

            #save their links
            for recipe in recipes:
                recipe_links.append(recipe.a["href"])

        #if there are no new pages
        else:
            break

In [74]:
recipe_info = []

#looping through recipe links
for new_url in recipe_links:
    
    recipe_dict = {}
    
    #reading info from recipe page
    new_request = urllib.request.Request(new_url,None,headers) #The assembled request
    new_response = urllib.request.urlopen(new_request)
    new_data = new_response.read()
    new_response.close()
    
    recipe_page = soup(new_data,"html.parser")
    
    #getting objects that contain details and ratings
    recipe_details = recipe_page.find("div",{"class":"tasty-recipes-details"})
    recipe_ratings = recipe_page.find("div",{"class":"tasty-recipes-ratings"})
    
    #if details and ratings are available
    if recipe_details and recipe_ratings:
        #recipe name
        recipe_dict["title"] = recipe_page.h1.text

        #recipe details - time, cuisine, category
        try:
            recipe_dict["time"] = recipe_details.find("span",{"class":"tasty-recipes-total-time"}).text
            recipe_dict["cuisine"] = recipe_details.find("span",{"class":"tasty-recipes-cuisine"}).text
            recipe_dict["category"] = recipe_details.find("span",{"class":"tasty-recipes-category"}).text
        except:
            recipe_dict["cuisine"] = None
            recipe_dict["category"] = None

        #recipe ratings
        recipe_dict["rating"] = recipe_ratings.find("span",{"class":"average"}).text
        
        recipe_dict["url"] = new_url

        #adding dictionary of recipe info to list
        recipe_info.append(recipe_dict)

In [78]:
recipes_df = pd.DataFrame(recipe_info)
recipes_df["rating"] = recipes_df["rating"].astype("float")

Recipes for a Mexican main dish with a rating of 4.5 stars or higher 

In [85]:
recipes_df[(recipes_df["cuisine"]=="Mexican") & (recipes_df["rating"]>=4.5) & (recipes_df["category"]=="Main")].sort_values("rating",ascending=False)[:3]


,title,time,cuisine,category,rating,url
455,Fresh Arugula and Black Bean Tacos with Pickle...,30 minutes,Mexican,Main,5.0,https://cookieandkate.com/fresh-arugula-and-bl...
527,Chipotle-Glazed Delicata Squash,1 hour 15 minutes,Mexican,Main,5.0,https://cookieandkate.com/chipotle-glazed-squash/
58,"Crispy Mushroom, Spinach and Avocado Quesadillas",20 minutes,Mexican,Main,4.9,https://cookieandkate.com/crispy-mushroom-spin...


In [88]:
recipes_df.to_csv("./cookieandkaterecipes.csv")